In [5]:
import argparse
import httplib2
import requests
import pandas as pd 

from collections import defaultdict
from dateutil import relativedelta
from googleapiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [6]:
# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "165277451909-549djem84js33jh3thql01iccovj6hsl.apps.googleusercontent.com"
CLIENT_SECRET = "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"

# Define Oath scopes with read only access
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"

parser = argparse.ArgumentParser(
    formatter_class=argparse.RawDescriptionHelpFormatter,
    parents=[tools.argparser])
flags = parser.parse_args([])
 
flow = client.OAuth2WebServerFlow(client_id=CLIENT_ID,
                           client_secret=CLIENT_SECRET,
                           scope=OAUTH_SCOPE)
 
storage = file.Storage('searchconsole.dat')
 
credentials = storage.get()
if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage, flags)
 
# Create an httplib2.Http object and authorize it with our credentials
http = credentials.authorize(http=httplib2.Http())

In [7]:
webmasters_service = build('searchconsole', 'v1', http)
webmasters_service

How to get a list of Sites we have in GSC Account?

In [7]:
# Get a list of site in my Google Search Console Account
site_list = webmasters_service.sites().list().execute()
site_list

{'siteEntry': [{'siteUrl': 'https://www.samsung.com/tr/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/ae_ar/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/ba/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.ascentkorea.com/',
   'permissionLevel': 'siteUnverifiedUser'},
  {'siteUrl': 'https://www.samsung.com/se/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/au/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/fr/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/ge/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/hk/',
   'permissionLevel': 'siteRestrictedUser'},
  {'siteUrl': 'https://www.samsung.com/latin_en/',
   'permissionLevel': 'siteRestrict

How to get Google Search Console Analytics Ddata using API?

How to use While Loop to get more than 25,000 Rows of Data

In [25]:
import pandas as pd 

# the website we want to get the data for
website = ["https://www.samsung.com/uk/", "https://www.samsung.com/us/"]
startdate = ['2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']
enddate = ['2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29']

# create an empty list to store the rows from response
all_response = []

#define a startRow
startRow = 0

for property in website:
    for i in range(len(startdate)):
        while (startRow == 0) or (startRow%25000 == 0):
            # buile a requst body
            request_body = {
                "startDate" : startdate[i],
                "endDate" : enddate[i],
                "dimensions" : ['Date'],
                "rowLimit": 25000,
                "dataState" : "final",
                "startRow" : startRow
                }
            #get gsc response
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        
            #update the rows
            startRow = startRow + len(response_data['rows'])
            print("fetched up to " + str(startRow) + " rows of data")
        
       #for loop to save all the rows in all_responses
        for row in response_data['rows']:
            all_response.append(row)

        data_for_df = []

        for each in all_response:
            temp = []
            
            #date
            temp.append(each['keys'][0])
            
            #clicks
            temp.append(each['clicks'])
            
            #impressions
            temp.append(each['impressions'])
            
            #ctr
            temp.append(each['ctr'])
            
            #position
            temp.append(each['position'])
        
            #property
            temp.append(website)
            
            data_for_df.append(temp)

            df = pd.DataFrame(data_for_df, columns=['date', 'clicks', 'impressions', 'ctr', 'position', 'property'])
            
            df.to_csv(f'{property.split('/')[3] + '-' +startdate[i]+'.csv'}')

fetched up to 31 rows of data


Build a Dataframe using Pandas

In [14]:
all_response[0]

{'keys': ['2022-11-01', 'https://www.samsung.com/uk/', 'samsung'],
 'clicks': 5882,
 'impressions': 18714,
 'ctr': 0.31431014213957464,
 'position': 1.0500160307790958}

In [7]:
import pandas as pd

In [24]:
#empty list to build a dataframe
data_for_df = []

for each in all_response:
    temp = []
    
    #date
    temp.append(each['keys'][0])
    
    #page
    temp.append(each['keys'][1])
    
    #query
    temp.append(each['keys'][2])
    
    #clicks
    temp.append(each['clicks'])
    
    #impressions
    temp.append(each['impressions'])
    
    #ctr
    temp.append(each['ctr'])
    
    #position
    temp.append(each['position'])

    #property
    temp.append(website)
    
    data_for_df.append(temp)

In [25]:
data_for_df[0]

['2022-11-01',
 'https://www.samsung.com/uk/',
 'samsung',
 5882,
 18714,
 0.31431014213957464,
 1.0500160307790958,
 'https://www.samsung.com/uk/']

In [26]:
df = pd.DataFrame(data_for_df, columns=['date', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'property'])

In [27]:
df

,date,page,query,clicks,impressions,ctr,position,property
0,2022-11-01,https://www.samsung.com/uk/,samsung,5882,18714,0.314310,1.050016,https://www.samsung.com/uk/
1,2022-11-01,https://www.samsung.com/uk/,samsung uk,1769,3290,0.537690,1.000000,https://www.samsung.com/uk/
2,2022-11-01,https://www.samsung.com/uk/smartphones/galaxy-...,samsung s22 ultra,430,4225,0.101775,1.039290,https://www.samsung.com/uk/
3,2022-11-01,https://www.samsung.com/uk/smartphones/galaxy-...,samsung s22,380,3829,0.099243,1.000261,https://www.samsung.com/uk/
4,2022-11-01,https://www.samsung.com/uk/smartphones/galaxy-...,samsung a13,322,16767,0.019204,4.520666,https://www.samsung.com/uk/
...,...,...,...,...,...,...,...,...
36498,2022-11-01,https://www.samsung.com/uk/watches/galaxy-watc...,samsung smart watch,0,255,0.000000,1.000000,https://www.samsung.com/uk/
36499,2022-11-01,https://www.samsung.com/uk/watches/galaxy-watc...,samsung watches,0,69,0.000000,1.000000,https://www.samsung.com/uk/
36500,2022-11-01,https://www.samsung.com/uk/watches/galaxy-watc...,galaxy watch,0,67,0.000000,1.000000,https://www.samsung.com/uk/
36501,2022-11-01,https://www.samsung.com/uk/watches/galaxy-watc...,samsung galaxy watch,0,62,0.000000,1.000000,https://www.samsung.com/uk/


Export CSV or Excel

In [28]:
df.to_csv('test.csv')

In [8]:
website = ["https://www.samsung.com/uk/", "https://www.samsung.com/us/"]
website[0].split('/')[3]

'uk'

In [15]:
website = ["https://www.samsung.com/uk/", "https://www.samsung.com/us/"]
startdate = ['2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']

print(f'{website[0].split('/')[3] + '-' +startdate[0]}.csv')

uk-2022-12-01.csv


In [20]:
startdate = ['2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']

for i in range(len(startdate)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [21]:
startdate = ['2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']

for i in startdate:
    print(i)

2022-12-01
2023-01-01
2023-02-01
2023-03-01
2023-04-01
2023-05-01
2023-06-01
2023-07-01
2023-08-01
2023-09-01
2023-10-01
2023-11-01
2023-12-01
2024-01-01
2024-02-01


# PAGE, DATE

In [4]:
import pandas as pd 

# the website we want to get the data for
websites = ["https://www.samsung.com/in/", "https://www.samsung.com/sg/", "https://www.samsung.com/au/", "https://www.samsung.com/ca/", "https://www.samsung.com/sec/", "https://www.samsung.com/jp/", "https://www.samsung.com/sg/"]
startdate = ['2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']
enddate = ['2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29']

for property in websites:
    for i in range(len(startdate)):
        all_response = []  # Initialize all_response for each website
        
        startRow = 0

        while (startRow == 0) or (startRow % 25000 == 0):
            request_body = {
                "startDate": startdate[i],
                "endDate": enddate[i],
                "dimensions": ['Date', 'Page'],
                "rowLimit": 25000,
                "dataState": "final",
                "startRow": startRow
            }
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            startRow += len(response_data['rows'])
            print("fetched up to " + str(startRow) + " rows of data")

            all_response.extend(response_data['rows'])

        data_for_df = []

        for row in all_response:
            temp = []
            temp.append(row['keys'][0])  # date
            temp.append(row['keys'][1])  #page
            temp.append(row['clicks'])    # clicks
            temp.append(row['impressions'])  # impressions
            temp.append(row['ctr'])       # ctr
            temp.append(row['position'])  # position
            temp.append(property)         # website
            data_for_df.append(temp)

        df = pd.DataFrame(data_for_df, columns=['date', 'page', 'clicks', 'impressions', 'ctr', 'position', 'website'])
        df.to_csv(f'{property.split("/")[-2]}-{startdate[i]}-date-page.csv', index=False)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 134933 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 134665 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 120037 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 126990 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 117741 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of da

KeyError: 'rows'

# Date, Page, Query

In [8]:
import pandas as pd 

website = "https://www.samsung.com/uk/"
all_response = []  # Initialize all_response for each website
        
startRow = 0

while (startRow == 0) or (startRow % 25000 == 0):
    request_body = {
        "startDate": '2024-01-01',
        "endDate": '2024-01-31',
        "dimensions": ['Date', 'Page', 'Query'],
        "rowLimit": 25000,
        "dataState": "final",
        "startRow": startRow
    }
    response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()
    startRow += len(response_data['rows'])
    print("fetched up to " + str(startRow) + " rows of data")

    all_response.extend(response_data['rows'])

data_for_df = []

for row in all_response:
    temp = []
    temp.append(row['keys'][0])  # date
    temp.append(row['keys'][1])  #page
    temp.append(row['keys'][2])  #query
    temp.append(row['clicks'])    # clicks
    temp.append(row['impressions'])  # impressions
    temp.append(row['ctr'])       # ctr
    temp.append(row['position'])  # position
    temp.append(property)         # website
    data_for_df.append(temp)

df = pd.DataFrame(data_for_df, columns=['date', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'website'])
df.to_csv(f'{website.split("/")[-2]}-2024-01-01-date-page-query.csv', index=False)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 225000 rows of data
fetched up to 250000 rows of data
fetched up to 275000 rows of data
fetched up to 300000 rows of data
fetched up to 325000 rows of data
fetched up to 350000 rows of data
fetched up to 375000 rows of data
fetched up to 400000 rows of data
fetched up to 425000 rows of data
fetched up to 450000 rows of data
fetched up to 475000 rows of data
fetched up to 500000 rows of data
fetched up to 525000 rows of data
fetched up to 550000 rows of data
fetched up to 575000 rows of data
fetched up to 600000 rows of data
fetched up to 625000 rows of data
fetched up to 650000 rows of data
fetched up to 675000 rows of data
fetched up to 700000 rows of data
fetched up to 725000 rows of data
fetched up to 750

데이터가 잘리지는 않았을까? > 이걸 태블로가 아니라 파이썬에서 바로 확인할 수 있게되면 좋겠다..
GSC에서 뱉어주는 최대 행수가 몇인지 확인필요

In [ ]:
import pandas as pd 

# the website we want to get the data for
websites = ["https://www.samsung.com/in/", "https://www.samsung.com/sg/", "https://www.samsung.com/au/", "https://www.samsung.com/ca/", "https://www.samsung.com/sec/", "https://www.samsung.com/jp/", "https://www.samsung.com/sg/"]
startdate = ['2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']
enddate = ['2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29']

for property in websites:
    for i in range(len(startdate)):
        all_response = []  # Initialize all_response for each website
        
        startRow = 0

        while (startRow == 0) or (startRow % 25000 == 0):
            request_body = {
                "startDate": startdate[i],
                "endDate": enddate[i],
                "dimensions": ['Date', 'Page', 'Query'],
                "rowLimit": 25000,
                "dataState": "final",
                "startRow": startRow
            }
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            startRow += len(response_data['rows'])
            print("fetched up to " + str(startRow) + " rows of data")

            all_response.extend(response_data['rows'])

        data_for_df = []

        for row in all_response:
            temp = []
            temp.append(row['keys'][0])  # date
            temp.append(row['keys'][1])  #page
            temp.append(row['keys'][2])  #query
            temp.append(row['clicks'])    # clicks
            temp.append(row['impressions'])  # impressions
            temp.append(row['ctr'])       # ctr
            temp.append(row['position'])  # position
            temp.append(property)         # website
            data_for_df.append(temp)

        df = pd.DataFrame(data_for_df, columns=['date', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'website'])
        df.to_csv(f'{property.split("/")[-2]}-{startdate[i]}-date-page-query.csv', index=False)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 225000 rows of data
fetched up to 250000 rows of data
fetched up to 275000 rows of data
fetched up to 300000 rows of data
fetched up to 325000 rows of data
fetched up to 350000 rows of data
fetched up to 375000 rows of data
fetched up to 400000 rows of data
fetched up to 425000 rows of data
fetched up to 450000 rows of data
fetched up to 475000 rows of data
fetched up to 500000 rows of data
fetched up to 525000 rows of data
fetched up to 550000 rows of data
fetched up to 575000 rows of data
fetched up to 600000 rows of data
fetched up to 625000 rows of data
fetched up to 650000 rows of data
fetched up to 675000 rows of data
fetched up to 700000 rows of data
fetched up to 725000 rows of data
fetched up to 750

# Date, Page(당월데이터)

In [8]:
website = ["https://www.samsung.com/uk/", "https://www.samsung.com/us/", "https://www.samsung.com/in/", "https://www.samsung.com/sg/", "https://www.samsung.com/au/", "https://www.samsung.com/ca/", "https://www.samsung.com/sec/", "https://www.samsung.com/jp/", "https://www.samsung.com/sg/"]
STARTDATE = '2024-03-01'
ENDDATE = '2024-03-26'

for property in website:
    all_response = []
    startRow = 0
    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate" : STARTDATE,
            "endDate" : ENDDATE,
            "dimensions" : ['Date', 'Page'],
            "rowLimit" : 25000,
            "dateState" : "final",
            "startRow" : startRow
        }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        startRow += len(response_data['rows'])
        print("fetched up to " + str(startRow) + " rows of data")
        all_response.extend(response_data['rows'])
    
    data_for_df = []
    
    for row in all_response:
        temp = []
        temp.append(row['keys'][0])  #date
        temp.append(row['keys'][1])  #page
        temp.append(row['clicks'])
        temp.append(row['impressions'])
        temp.append(row['ctr'])
        temp.append(row['position'])
        temp.append(property)
        data_for_df.append(temp)
    
    df = pd.DataFrame(data_for_df, columns = ['date', 'page', 'clicks', 'impressions', 'ctr', 'position', 'website'])
    df.to_csv(f'{property.split("/")[-2]}-{STARTDATE}-to-{df['date'].max()}-date-page.csv', index=False)


fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 106318 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 94999 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 91994 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 96229 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 100626 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 94673 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetch